In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 模型花园 TFVision 图像目标检测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>

  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_tfvision_image_object_detection.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在 Vertex AI 工作台中打开
    </a>
  </td>
</table>

注意：此笔记本已在以下环境中进行了测试：

- Python 版本 = 3.9

## 概览

本笔记本演示了如何在Vertex AI模型花园中使用[TFVision](https://github.com/tensorflow/models/blob/master/official/vision/MODEL_GARDEN.md)。

### 目标

* 训练新模型
  * 将输入数据转换为训练格式
  * 创建[超参数调整作业](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)来训练新模型
  * 查找并导出最佳模型

* 测试训练模型
  * 上传模型到模型注册表
  * 部署上传的模型
  * 进行预测

* 清理资源

### 成本

此教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing
Calculator](https://cloud.google.com/products/calculator/)
根据您的预计使用量生成成本估算。

在你开始之前

仅限Colab
如果您使用Workbench，请运行以下命令并跳过本部分。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform

    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

    from google.colab import auth as google_auth

    google_auth.authenticate_user()

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，都需要按照以下步骤进行操作。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用额度，可用于支付计算/存储费用。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. 如果您正在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

5. 在下面的单元格中输入您的项目ID。然后运行单元格，确保Cloud SDK在本笔记本中的所有命令中都使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
import os

from google.cloud import aiplatform

# The project and bucket are for experiments below.
PROJECT_ID = ""  # @param {type:"string"}
BUCKET_URI = ""  # @param {type:"string"}

# You can choose a region from https://cloud.google.com/about/locations.
# Only regions prefixed by "us", "asia", or "europe" are supported.
REGION = "us-central1"  # @param {type:"string"}
REGION_PREFIX = REGION.split("-")[0]
assert REGION_PREFIX in (
    "us",
    "europe",
    "asia",
), f'{REGION} is not supported. It must be prefixed by "us", "asia", or "europe".'

! gcloud config set project $PROJECT_ID

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
CHECKPOINT_BUCKET = os.path.join(BUCKET_URI, "ckpt")

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Download config files.
CONFIG_DIR = os.path.join(BUCKET_URI, "config")
! wget https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/retinanet/coco_spinenet49_gpu_multiworker_mirrored.yaml
! gsutil cp coco_spinenet49_gpu_multiworker_mirrored.yaml $CONFIG_DIR/

! wget https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/retinanet/coco_spinenet96_gpu_multiworker_mirrored.yaml
! gsutil cp coco_spinenet96_gpu_multiworker_mirrored.yaml $CONFIG_DIR/

! wget https://raw.githubusercontent.com/tensorflow/models/master/official/vision/configs/experiments/retinanet/coco_spinenet143_gpu_multiworker_mirrored.yaml
! gsutil cp coco_spinenet143_gpu_multiworker_mirrored.yaml $CONFIG_DIR/

! wget https://raw.githubusercontent.com/tensorflow/models/master/official/projects/yolo/configs/experiments/yolov4/detection/scaled_yolov4_1280_gpu.yaml
! gsutil cp scaled_yolov4_1280_gpu.yaml $CONFIG_DIR/

! wget https://raw.githubusercontent.com/tensorflow/models/master/official/projects/yolo/configs/experiments/yolov7/detection/yolov7_gpu.yaml
! gsutil cp yolov7_gpu.yaml $CONFIG_DIR/

### 定义常量

In [ ]:
OBJECTIVE = "iod"

# Data converter constants.
DATA_CONVERTER_JOB_PREFIX = "data_converter"
DATA_CONVERTER_CONTAINER = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/data-converter"
DATA_CONVERTER_MACHINE_TYPE = "n1-highmem-8"


# Training constants.
TRAINING_JOB_PREFIX = "train"
TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/tfvision-oss"
TRAIN_MACHINE_TYPE = "n1-highmem-16"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 2
TRAIN_SPINENET49_CONFIG = os.path.join(
    CONFIG_DIR, "coco_spinenet49_gpu_multiworker_mirrored.yaml"
)
TRAIN_SPINENET96_CONFIG = os.path.join(
    CONFIG_DIR, "coco_spinenet96_gpu_multiworker_mirrored.yaml"
)
TRAIN_SPINENET143_CONFIG = os.path.join(
    CONFIG_DIR, "coco_spinenet143_gpu_multiworker_mirrored.yaml"
)
TRAIN_YOLOV4_CONFIG = os.path.join(CONFIG_DIR, "scaled_yolov4_1280_gpu.yaml")
TRAIN_YOLOV7_CONFIG = os.path.join(CONFIG_DIR, "yolov7_gpu.yaml")

# Evaluation constants.
EVALUATION_METRIC = "AP50"

# Export constants.
EXPORT_JOB_PREFIX = "export"
EXPORT_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/vertex-vision-model-garden-dockers/tfvision-model-export"
EXPORT_MACHINE_TYPE = "n1-highmem-8"

# Prediction constants.
# You can deploy models with
#   pre-build-dockers: https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers.
#   and optimized tensorflow runtime dockers: https://cloud.google.com/vertex-ai/docs/predictions/optimized-tensorflow-runtime.
# The example in this notebook uses optimized tensorflow runtime dockers.
# You can adjust accelerator types and machine types to get faster predictions.
PREDICTION_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-11:latest"
PREDICTION_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
PREDICTION_MACHINE_TYPE = "n1-standard-4"
UPLOAD_JOB_PREFIX = "upload"
DEPLOY_JOB_PREFIX = "deploy"

### 定义常用的库

In [ ]:
import base64
import json
import os
from datetime import datetime
from io import BytesIO
from typing import Dict, List, Union

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import yaml
from google.cloud import aiplatform
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from PIL import Image, ImageColor, ImageDraw, ImageFont


def get_job_name_with_datetime(prefix: str):
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def predict_custom_trained_model(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": f"{location}-aiplatform.googleapis.com"}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    return response.predictions, response.deployed_model_id


def load_img(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return Image.fromarray(np.uint8(img)).convert("RGB")


def display_image(image):
    _ = plt.figure(figsize=(20, 15))
    plt.grid(False)
    plt.imshow(image)


def get_prediction_instances(test_filepath, new_width=-1):
    if new_width <= 0:
        with tf.io.gfile.GFile(test_filepath, "rb") as input_file:
            encoded_string = base64.b64encode(input_file.read()).decode("utf-8")
    else:
        img = load_img(test_filepath)
        width, height = img.size
        print("original input image size: ", width, " , ", height)
        new_height = int(height * new_width / width)
        new_img = img.resize((new_width, new_height))
        print("resized input image size: ", new_width, " , ", new_height)
        buffered = BytesIO()
        new_img.save(buffered, format="JPEG")
        encoded_string = base64.b64encode(buffered.getvalue()).decode("utf-8")

    instances = [
        {
            "encoded_image": {"b64": encoded_string},
        }
    ]
    return instances


def get_label_map(label_map_yaml_filepath):
    with tf.io.gfile.GFile(label_map_yaml_filepath, "rb") as input_file:
        label_map = yaml.safe_load(input_file.read())
    return label_map


def get_best_trial(model_dir, max_trial_count, evaluation_metric):
    best_trial_dir = ""
    best_trial_evaluation_results = {}
    best_performance = -1

    for i in range(max_trial_count):
        current_trial = i + 1
        current_trial_dir = os.path.join(model_dir, "trial_" + str(current_trial))
        current_trial_best_ckpt_dir = os.path.join(current_trial_dir, "best_ckpt")
        current_trial_best_ckpt_evaluation_filepath = os.path.join(
            current_trial_best_ckpt_dir, "info.json"
        )
        with tf.io.gfile.GFile(current_trial_best_ckpt_evaluation_filepath, "rb") as f:
            eval_metric_results = json.load(f)
            current_performance = eval_metric_results[evaluation_metric]
            if current_performance > best_performance:
                best_performance = current_performance
                best_trial_dir = current_trial_dir
                best_trial_evaluation_results = eval_metric_results
    return best_trial_dir, best_trial_evaluation_results


def draw_bounding_box_on_image(
    image, ymin, xmin, ymax, xmax, color, font, thickness=4, display_str_list=()
):
    """Adds a bounding box to an image."""
    draw = ImageDraw.Draw(image)
    im_width, im_height = image.size
    (left, right, top, bottom) = (
        xmin * im_width,
        xmax * im_width,
        ymin * im_height,
        ymax * im_height,
    )
    draw.line(
        [(left, top), (left, bottom), (right, bottom), (right, top), (left, top)],
        width=thickness,
        fill=color,
    )

    # If the total height of the display strings added to the top of the bounding
    # box exceeds the top of the image, stack the strings below the bounding box
    # instead of above.
    display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
    # Each display_str has a top and bottom margin of 0.05x.
    total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

    if top > total_display_str_height:
        text_bottom = top
    else:
        text_bottom = top + total_display_str_height
    # Reverse list and print from bottom to top.
    for display_str in display_str_list[::-1]:
        text_width, text_height = font.getsize(display_str)
        margin = np.ceil(0.05 * text_height)
        draw.rectangle(
            [
                (left, text_bottom - text_height - 2 * margin),
                (left + text_width, text_bottom),
            ],
            fill=color,
        )
        draw.text(
            (left + margin, text_bottom - text_height - margin),
            display_str,
            fill="black",
            font=font,
        )
        text_bottom -= text_height - 2 * margin


def draw_boxes(image, boxes, class_names, scores, max_boxes=40, min_score=0.05):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    colors = list(ImageColor.colormap.values())
    try:
        font = ImageFont.truetype(
            "/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf", 25
        )
    except OSError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(len(boxes), max_boxes)):
        if scores[i] >= min_score:
            ymin, xmin, ymax, xmax = boxes[i]
            display_str = "{}: {}%".format(class_names[i], int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            draw_bounding_box_on_image(
                image,
                ymin,
                xmin,
                ymax,
                xmax,
                color,
                font,
                display_str_list=[display_str],
            )
    return image


def upload_checkpoint_to_gcs(checkpoint_url):
    filename = os.path.basename(checkpoint_url)
    checkpoint_name = filename.replace(".tar.gz", "")
    print("Download checkpoint from", checkpoint_url, "and store to", CHECKPOINT_BUCKET)
    ! wget $checkpoint_url -O $filename
    ! mkdir -p $checkpoint_name
    ! tar -xvzf $filename -C $checkpoint_name

    # Search for relative path to the checkpoint.
    checkpoint_path = None
    for root, dirs, files in os.walk(checkpoint_name):
        for file in files:
            if file.endswith(".index"):
                checkpoint_path = os.path.join(root, os.path.splitext(file)[0])
                checkpoint_path = os.path.relpath(checkpoint_path, checkpoint_name)
                break

    ! gsutil cp -r $checkpoint_name $CHECKPOINT_BUCKET/
    checkpoint_uri = os.path.join(CHECKPOINT_BUCKET, checkpoint_name, checkpoint_path)
    print("Checkpoint uploaded to", checkpoint_uri)
    return checkpoint_uri

## 训练新模型
该部分展示了如何训练新模型。
1. 将输入数据转换为训练格式
2. 创建超参数调整作业以训练新模型
3. 查找并导出最佳模型

如果您已经训练了模型，请转到“测试训练过的模型”部分。

### 为训练准备输入数据

按照[此处](https://cloud.google.com/vertex-ai/docs/image-data/classification/prepare-data)描述的格式准备数据，然后将其转换为以下训练格式：

* `input_file_path`：用于准备数据的输入文件路径。
* `input_file_type`：输入文件类型，例如 csv 或 jsonl。
* `split_ratio`：要拆分为训练/验证/测试的数据比例。
* `num_shard`：用于训练/验证/测试的分片数量。
* `output_dir`：包含准备好的训练/测试/验证数据的输出目录。

In [ ]:
# This job will convert input data as training format, with given split ratios
# and number of shards on train/test/validation.
data_converter_job_name = get_job_name_with_datetime(
    DATA_CONVERTER_JOB_PREFIX + "_" + OBJECTIVE
)

input_file_path = ""  # @param {type:"string"}
input_file_type = "csv"  # @param ['csv', 'jsonl', 'coco_json']
split_ratio = "0.8,0.1,0.1"
num_shard = "10,10,10"
data_converter_output_dir = os.path.join(BUCKET_URI, data_converter_job_name)


worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": DATA_CONVERTER_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": DATA_CONVERTER_CONTAINER,
            "command": [],
            "args": [
                "--input_file_path=%s" % input_file_path,
                "--input_file_type=%s" % input_file_type,
                "--objective=%s" % OBJECTIVE,
                "--num_shard=%s" % num_shard,
                "--split_ratio=%s" % split_ratio,
                "--output_dir=%s" % data_converter_output_dir,
            ],
        },
    }
]

data_converter_custom_job = aiplatform.CustomJob(
    display_name=data_converter_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

data_converter_custom_job.run()

input_train_data_path = os.path.join(data_converter_output_dir, "train.tfrecord*")
input_validation_data_path = os.path.join(data_converter_output_dir, "val.tfrecord*")
label_map_path = os.path.join(data_converter_output_dir, "label_map.yaml")
print("input_train_data_path for training: ", input_train_data_path)
print("input_validation_data_path for training: ", input_validation_data_path)
print("label_map_path for prediction: ", label_map_path)

使用Vertex AI SDK创建并运行具有超参数调整功能的Vertex AI Model Garden Training Dockers自定义作业。

#### 定义以下规格

* `worker_pool_specs`: 字典指定机器类型和Docker镜像。这个例子定义了一个使用一个 `n1-highmem-16` 机器和两个 `NVIDIA_TESLA_V100` GPU 的单节点集群。
* `parameter_spec`: 字典指定要优化的参数。字典键是分配给训练应用代码中每个超参数的命令行参数的字符串，字典值是参数规范。参数规范包括超参数的类型、最小/最大值和尺度。
* `metric_spec`: 字典指定要优化的指标。字典键是你在训练应用代码中设置的 `hyperparameter_metric_tag`，值是优化目标。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

label_map = get_label_map(label_map_path)
num_classes = len(label_map["label_map"]) + 1

# Input train and validation datasets can be found from the section above
# `Convert input data for training`.
# Set prepared datasets if exists.
# input_train_data_path = ''
# input_validation_data_path = ''

# Refer to https://github.com/tensorflow/models/blob/master/official/vision/MODEL_GARDEN.md
# for more model details.
experiment = "retinanet_spinenet96"  # @param ['retinanet_spinenet49', "retinanet_spinenet96", 'retinanet_spinenet143', 'scaled_yolo_v4', 'yolov7']

train_job_name = get_job_name_with_datetime(TRAINING_JOB_PREFIX + "_" + OBJECTIVE)
model_dir = os.path.join(BUCKET_URI, train_job_name)

# The arguments here are mainly for test purposes. Please update them
# to get better performances.
common_args = {
    "input_train_data_path": input_train_data_path,
    "input_validation_data_path": input_validation_data_path,
    "objective": OBJECTIVE,
    "model_dir": model_dir,
    "num_classes": num_classes,
    "global_batch_size": 2,
    "prefetch_buffer_size": 6,
    "train_steps": 2000,
    "input_size": "1024,1024",
}

experiment_container_args_dict = {
    # retinanet_spinenet49 experiment args.
    "retinanet_spinenet49": dict(
        common_args,
        **{
            "experiment": "retinanet_spinenet_coco",
            "config_file": TRAIN_SPINENET49_CONFIG,
            "anchor_size": 4,
        },
    ),
    # retinanet_spinenet96 experiment args.
    "retinanet_spinenet96": dict(
        common_args,
        **{
            "experiment": "retinanet_spinenet_coco",
            "config_file": TRAIN_SPINENET96_CONFIG,
            "anchor_size": 4,
        },
    ),
    # retinanet_spinenet143 experiment args.
    "retinanet_spinenet143": dict(
        common_args,
        **{
            "experiment": "retinanet_spinenet_coco",
            "config_file": TRAIN_SPINENET143_CONFIG,
            "anchor_size": 4,
        },
    ),
    # scaled_yolo_v4 experiment args.
    "scaled_yolo_v4": dict(
        common_args,
        **{
            "experiment": "scaled_yolo",
            "config_file": TRAIN_YOLOV4_CONFIG,
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/yolo/scaled-yolov4/scaled-yolov4-l-p6-i1280.tar.gz",
            "input_size": "1280,1280",
        },
    ),
    # yolov7 experiment args.
    "yolov7": dict(
        common_args,
        **{
            "experiment": "coco_yolov7",
            "config_file": TRAIN_YOLOV7_CONFIG,
            "init_checkpoint": "https://storage.googleapis.com/tf_model_garden/vision/yolo/yolov7/yolov7.tar.gz",
            "input_size": "640,640",
        },
    ),
}
experiment_container_args = experiment_container_args_dict[experiment]

# Copy checkpoint to GCS bucket if specified.
init_checkpoint = experiment_container_args.get("init_checkpoint")
if init_checkpoint:
    experiment_container_args["init_checkpoint"] = upload_checkpoint_to_gcs(
        init_checkpoint
    )
if "yolov7" in experiment:
    TRAIN_CONTAINER_URI = f"{REGION_PREFIX}-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/tfvision-oss-v2"

params_override = "runtime.num_gpus=%s" % TRAIN_NUM_GPU
eval_params_override = "runtime.num_gpus=1,runtime.distribution_strategy=mirrored"
worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "args": [
                "--mode=train",
                "--params_override=%s" % params_override,
            ]
            + ["--{}={}".format(k, v) for k, v in experiment_container_args.items()],
        },
    },
    {},
    {},
    {
        "machine_spec": {
            "machine_type": "n1-highmem-4",
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            "accelerator_count": 1,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "args": [
                "--mode=continuous_eval",
                "--params_override=%s" % eval_params_override,
            ]
            + ["--{}={}".format(k, v) for k, v in experiment_container_args.items()],
        },
    },
]

metric_spec = {"model_performance": "maximize"}

LEARNING_RATES = [0.001, 0.01]
# Models will be trained with each learning rate separately and max trial count is the number of learning rates.
MAX_TRIAL_COUNT = len(LEARNING_RATES)
parameter_spec = {
    "learning_rate": hpt.DiscreteParameterSpec(values=LEARNING_RATES, scale="linear"),
}

print(worker_pool_specs, metric_spec, parameter_spec)

#### 运行超参数调整任务
* `max_trial_count`: 设置服务运行的试验次数上限。建议的做法是从较少的试验次数开始，了解选择的超参数对结果的影响程度，然后再逐步增加试验次数。

* `parallel_trial_count`: 如果使用并行试验，服务会提供多个训练处理集群。在创建作业时指定的工作池规范将用于每个单独的训练集群。增加并行试验的数量可以减少超参数调整作业所需的时间；然而，这可能会降低作业的效果。这是因为默认的调整策略使用先前试验的结果来指导后续试验中值的分配。

* `search_algorithm`: 可用的搜索算法有网格、随机或默认（无）。默认选项应用贝叶斯优化来搜索可能超参数值的空间，是推荐的算法。

点击输出中生成的链接，在云控制台中查看您的运行情况。

In [ ]:
train_custom_job = aiplatform.CustomJob(
    display_name=train_job_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=train_job_name,
    custom_job=train_custom_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=MAX_TRIAL_COUNT,
    parallel_trial_count=1,
    project=PROJECT_ID,
    search_algorithm=None,
)

train_hpt_job.run()

print("experiment is: ", experiment)
print("model_dir is: ", model_dir)

将最佳模型导出为TF Saved Model格式

In [ ]:
# This job will export models from TF checkpoints to TF saved model format.
# model_dir is from the section above.
best_trial_dir, best_trial_evaluation_results = get_best_trial(
    model_dir, MAX_TRIAL_COUNT, EVALUATION_METRIC
)
print("best_trial_dir: ", best_trial_dir)
print("best_trial_evaluation_results: ", best_trial_evaluation_results)

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": EXPORT_MACHINE_TYPE,
        },
        "replica_count": 1,
        "container_spec": {
            "image_uri": EXPORT_CONTAINER_URI,
            "command": [],
            "args": [
                "--objective=%s" % OBJECTIVE,
                "--input_image_size=1024,1024",
                "--experiment=%s" % experiment_container_args["experiment"],
                "--config_file=%s/params.yaml" % best_trial_dir,
                "--checkpoint_path=%s/best_ckpt" % best_trial_dir,
                "--export_dir=%s/best_model" % model_dir,
            ],
        },
    }
]

model_export_name = get_job_name_with_datetime(EXPORT_JOB_PREFIX + "_" + OBJECTIVE)
model_export_custom_job = aiplatform.CustomJob(
    display_name=model_export_name,
    project=PROJECT_ID,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)


model_export_custom_job.run()

print("best model is saved to: ", os.path.join(model_dir, "best_model"))

## 测试训练好的模型
本部分将展示如何使用训练好的模型进行测试。
1. 上传并部署模型
2. 运行预测结果

In [ ]:
# @title Upload and deploy models
# model_dir is from the section above.
trained_model_dir = os.path.join(model_dir, "best_model/saved_model")

upload_job_name = get_job_name_with_datetime(UPLOAD_JOB_PREFIX + "_" + OBJECTIVE)

if "yolo" in experiment:
    SERVING_CONTAINER_ARGS = ["--allow_precompilation"]
else:
    SERVING_CONTAINER_ARGS = ["--allow_precompilation", "--allow_compression"]


serving_env = {
    "MODEL_ID": "tfvision-scaled-yolo",
    "DEPLOY_SOURCE": "notebook",
}

model = aiplatform.Model.upload(
    display_name=upload_job_name,
    artifact_uri=trained_model_dir,
    serving_container_image_uri=PREDICTION_CONTAINER_URI,
    serving_container_args=SERVING_CONTAINER_ARGS,
    serving_container_environment_variables=serving_env,
)

model.wait()

print("The uploaded model name is: ", upload_job_name)

deploy_model_name = get_job_name_with_datetime(DEPLOY_JOB_PREFIX + "_" + OBJECTIVE)
print("The deployed job name is: ", deploy_model_name)

endpoint = model.deploy(
    deployed_model_display_name=deploy_model_name,
    machine_type=PREDICTION_MACHINE_TYPE,
    traffic_split={"0": 100},
    accelerator_type=PREDICTION_ACCELERATOR_TYPE,
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

endpoint_id = endpoint.name
print("endpoint id is: ", endpoint_id)

In [ ]:
# @title Run predictions

# endpoint_id was generated in the section above (`Upload and deploy models`).
endpoint_id = endpoint.name

# The test image file path.
test_filepath = ""  # @param {type:"string"}
score_threshold = 0.2  # @param {type:"number"}
# If the input image is too large, we will resize it for prediction.
instances = get_prediction_instances(test_filepath, new_width=1000)

# The label map file was generated from the section above (`Convert input data for training`).
label_map = get_label_map(label_map_path)["label_map"]

predictions, _ = predict_custom_trained_model(
    project=PROJECT_ID, location=REGION, endpoint_id=endpoint_id, instances=instances
)

img = load_img(test_filepath)
detection_boxes = predictions[0]["detection_boxes"]
detection_scores = predictions[0]["detection_scores"]
detection_classes_as_text = []

for detection_class in predictions[0]["detection_classes"]:
    detection_classes_as_text.append(label_map[int(detection_class)])

img = draw_boxes(
    img,
    detection_boxes,
    detection_classes_as_text,
    detection_scores,
    min_score=score_threshold,
)
display_image(img)

清理

In [ ]:
# Undeploy model and delete endpoint.
endpoint.delete(force=True)
# Delete models.
model.delete()
# Delete custom and hpt jobs.
if data_converter_custom_job.list(filter=f'display_name="{data_converter_job_name}"'):
    data_converter_custom_job.delete()
if train_hpt_job.list(filter=f'display_name="{train_job_name}"'):
    train_hpt_job.delete()
if model_export_custom_job.list(filter=f'display_name="{model_export_name}"'):
    model_export_custom_job.delete()